In [3]:
import numpy as np

In [1]:
def _create_diff_tensor_a(minibatch_size, num_dxs):
    diff_tensor = np.zeros([minibatch_size,num_dxs,num_dxs,num_dxs])
    for n in range(minibatch_size):
        for i in range(num_dxs):
            for j in range(num_dxs):
                diff_tensor[n,i,i,j] +=1
                diff_tensor[n,i,j,i] -=1
    return diff_tensor

In [4]:
_create_diff_tensor_a(3,2)

array([[[[ 0.,  1.],
         [-1.,  0.]],

        [[ 0., -1.],
         [ 1.,  0.]]],


       [[[ 0.,  1.],
         [-1.,  0.]],

        [[ 0., -1.],
         [ 1.,  0.]]],


       [[[ 0.,  1.],
         [-1.,  0.]],

        [[ 0., -1.],
         [ 1.,  0.]]]])

In [5]:
def _test_diff_tensor_a(test_count=5):
    from random import randint
    for _ in range(test_count):
        minibatch_size = randint(0, 10)
        num_dxs = randint(0, 10)
        a = _create_diff_tensor_a(minibatch_size, num_dxs)
        for n in range(minibatch_size):
            for i in range(num_dxs):
                for j in range(num_dxs):
                    for k in range(num_dxs):
                        if k == i and i != j:
                            assert (a[n, k, i, j] == 1)
                        elif k == j and i != j:
                            assert (a[n, k, i, j] == -1)
                        else:
                            assert (a[n, k, i, j] == 0)

In [6]:
_test_diff_tensor_a()

In [82]:
def _create_reordered_diff_tensor_a(ddx_max_length, ddxs_extended):
    minibatch_size = len(ddxs_extended)
    num_dxs = len(ddxs_extended[0])
    assert ddx_max_length <= num_dxs, "ddx_max_length: %d, num_dxs: %d"%(ddx_max_length,num_dxs)
    inverted_ddxs = map(_invert_permutation, ddxs_extended)
    diff_tensor = np.zeros([minibatch_size,num_dxs,ddx_max_length,num_dxs])
    for n in range(minibatch_size):
        assert len(ddxs_extended[n]) == num_dxs,\
            "for each n, need len(ddxs_extended[n]) == len(ddxs_extended[0]), but \n"+\
            "len(ddxs_extended[%d])==%d! and len(ddxs_extended[0])==%d"\
            %(n,len(ddxs_extended[n]),len(ddxs_extended[0]))
        for i in range(num_dxs):
            for j in range(num_dxs):
                if inverted_ddxs[n][i] < ddx_max_length:
                    diff_tensor[n,i,inverted_ddxs[n][i],inverted_ddxs[n][j]] +=1
        for i in range(num_dxs):
            for j in range(num_dxs):
                if inverted_ddxs[n][j] < ddx_max_length:
                    diff_tensor[n,i,inverted_ddxs[n][j],inverted_ddxs[n][i]] -=1
    return diff_tensor

In [104]:
def _create_reordered_diff_tensor_a(ddx_max_length, ddxs_extended):
    minibatch_size = len(ddxs_extended)
    num_dxs = len(ddxs_extended[0])
    assert ddx_max_length <= num_dxs, "ddx_max_length: %d, num_dxs: %d"%(ddx_max_length,num_dxs)
    diff_tensor = np.zeros([minibatch_size,num_dxs,ddx_max_length,num_dxs])
    for n in range(minibatch_size):
        assert len(ddxs_extended[n]) == num_dxs,\
            "for each n, need len(ddxs_extended[n]) == len(ddxs_extended[0]), but \n"+\
            "len(ddxs_extended[%d])==%d! and len(ddxs_extended[0])==%d"\
            %(n,len(ddxs_extended[n]),len(ddxs_extended[0]))
        for i in range(ddx_max_length):
            for j in range(num_dxs):
                diff_tensor[n,ddxs_extended[n][i],i,j] +=1
        for i in range(num_dxs):
            for j in range(ddx_max_length):
                diff_tensor[n,ddxs_extended[n][i],j,i] -=1
    return diff_tensor

In [105]:
def _invert_permutation(perm):
    graph_of_perm = enumerate(perm)
    sorted_by_range = sorted(graph_of_perm,key = lambda pair:pair[1])
    inverse_permutation = map(lambda pair:pair[0],sorted_by_range)
    return inverse_permutation

In [106]:
_invert_permutation([2,3,1,4,0])

[4, 2, 0, 1, 3]

In [107]:
def _test_reordered_diff_tensor_a(test_count=5):
    from random import randint, shuffle
    for _ in range(test_count):
        minibatch_size = randint(1, 10)
        num_dxs = randint(0, 10)
        ddx_max_length = randint(0, num_dxs)
        ddxs_extended = [range(num_dxs) for _ in range(minibatch_size)]
        map(shuffle,ddxs_extended)
        a = _create_reordered_diff_tensor_a(ddx_max_length, ddxs_extended)
        assert a.shape == (minibatch_size,num_dxs,ddx_max_length,num_dxs),\
            "a.shape: (%d,%d,%d,%d)"%a.shape+\
            "\n(minibatch_size,num_dxs,ddx_max_length,num_dxs): (%d,%d,%d,%d)"\
            %(minibatch_size,num_dxs,ddx_max_length,num_dxs)
        a_0 = _create_diff_tensor_a(minibatch_size, num_dxs)
        for n in range(minibatch_size):
            for i in range(ddx_max_length):
                for j in range(num_dxs):
                    for k in range(num_dxs):
                        assert a_0[n, k, ddxs_extended[n][i], ddxs_extended[n][j]] == a[n, k, i, j], \
                            "diff_mat[n,k,%d,%d]: %r\n intended diff mat: %r" % (
                            i, j, a[n, k, :, :], a_0[n, k, :, :])

In [103]:
_test_reordered_diff_tensor_a()